<a href="https://colab.research.google.com/github/ffocode/Dashboard/blob/main/Plotly_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Dependencies
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
!pip install python-exchangeratesapi &> /dev/null
import requests
!pip install pdfplumber &> /dev/null
import pdfplumber
import re
from collections import namedtuple
from pandas.core.frame import DataFrame
!pip install --upgrade gspread &> /dev/null
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import datetime


context = ssl._create_unverified_context()

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}


def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records

def get_data_from_MAS(url):
    # Balance sheet
    url = url
    req = urllib.request.Request(url,headers=headers)
    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        path = url+ "&offset=" + str(offset)
        path_header = urllib.request.Request(path,headers=headers)
        with urllib.request.urlopen(path_header) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['offset'] <= int(data['result']['total']):
            offset += 100
        else:
            break

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records

    


print("dependencies ready")


dependencies ready


In [2]:
#@title # **1 - Monetary Base**
MBC = ' '

#Getting data from  7.21(Monetary Base – End of period figures) and store it as a pandas dataframe type
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

# cert_of_indebt - Certificates of Indebtedness

# gov_notes_coins_circulation - Government notes/coins in circulation

# aggr_balance_bf_disc_win - 	Aggregate Balance before Discount Window
# outstanding_efbn - Outstanding Exchange Fund Bills and Notes

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn'],var_name='Monetary Base', value_name='HK$ million' )

#Plot
fig = px.area(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Monetary Base')

#Plot Format
fig.update_layout(title_text="Monetary Base – End of period figures")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(legend= {'itemsizing': 'constant'})

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([  
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
      
    )
)

fig.show()

mbc_dfl.to_excel("Monetary Base.xlsx")  

In [3]:
#@title #2 - Money Multiplier = M3 / MB (HKD)
MSA = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money/supply-adjusted?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#There is a break in data series due to the inclusion of short-term Exchange Fund placements of less than one month since April 1997.

#Getting data from 2.21 (Money supply – Adjusted for foreign currency swap deposits) and store it as a pandas dataframe type
msa_df = pd.json_normalize(get_data_from_HKMA(MSA))

#m3_hkd- M3 HK$(Adjusted to include foreign currency swap deposits)

#Since the duration of m3_hkd and mb_bf_disc_win_total are not the same

#We merge them as a new dataset by the end_of_month

m3_hkd = msa_df[['end_of_month','m3_hkd']]

mb_tot = mbc_df[['end_of_month','mb_bf_disc_win_total']]

m3_hkd_Over_mb_tot = pd.merge(m3_hkd, mb_tot, on='end_of_month')

m3_hkd_Over_mb_tot['m3Overmb'] = (m3_hkd_Over_mb_tot['m3_hkd'] / m3_hkd_Over_mb_tot['mb_bf_disc_win_total'])

#Plot
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3Overmb", title='M3(hkd) / MB(Total)')

#Plot Format
# Set x-axis title
fig.update_xaxes(title_text="end_of_month")
# Set y-axis title
fig.update_yaxes(title_text="M3 / MB")

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([   
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)
fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)
fig.show()




In [4]:
#@title #3 - Loan to Deposits
BS = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/balance-sheet-ais?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

#Getting data from 3.9.1 (Balance sheet – Authorized institutions) and store it as a pandas dataframe type
bs_records_df = pd.json_normalize(get_data_from_HKMA(BS))

fig = make_subplots(rows=1, cols=3)

#Calculate the ratio

#Assets Loans and advances to customers HK$ - asst_loanadvtocust_hkd
#Liabilities Deposits from customers HK$  - asst_loanadvtocust_hkd
bs_records_df['LtoD_hkd'] = (bs_records_df['asst_loanadvtocust_hkd'] / bs_records_df['liab_depcust_hkd'])

#Assets Loans and advances to customers F.C. - asst_loanadvtocust_fc
#Liabilities Deposits from customers F.C.  - liab_depcust_fc
bs_records_df['LtoD_fc'] = (bs_records_df['asst_loanadvtocust_fc'] / bs_records_df['liab_depcust_fc'])

#Assets Loans and advances to customers Total - asst_loanadvtocust_total
#Liabilities Deposits from customers Total  - liab_depcust_total
bs_records_df['LtoD_total'] = (bs_records_df['asst_loanadvtocust_total'] / bs_records_df['liab_depcust_total'])


#Plot
fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_hkd"], mode='lines',
              name ='Loan to Deposits(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_fc"], mode='lines',
              name ='Loan to Deposits(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_total"], mode='lines',
              name ='Loan to Deposits (Total))'), row = 1, col=3)

#Plot Format
fig.update_xaxes(title_text="end_of_month")
fig.update_yaxes(title_text="Loan to Deposits")
fig.update_layout(title_text="Loan to Deposits")

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([   
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.update_layout(legend= {'itemsizing': 'constant'})


fig.show()

In [17]:
#@title #4 - Centaline index

gc = gspread.authorize(GoogleCredentials.get_application_default())

#Format the Dashboard_CCI URL of Googler Sheet
#https://docs.google.com/spreadsheets/d/1uLWbeFENJkYf2ei9zbf4Mf5094rQ0m6unrGZJhwd6W4/edit#gid=0
wb = gc.open_by_key('1uLWbeFENJkYf2ei9zbf4Mf5094rQ0m6unrGZJhwd6W4')

#Open the CCI worksheet
ws_cci = wb.worksheet('CCI')
# get_all_values gives a list of rows.
rows = ws_cci.get_all_values()


# for m in month:
#  val.append(m[0:4] + "-" + m[5:7])
#  print(m[0:4])

#cell_list = ws_cci.range('A2:A271')
#cell_values = val

#for i, value in enumerate(cell_values):  #gives us a tuple of an index and value
#    cell_list[i].value = value    #use the index on cell_list and the val from cell_values

#ws_cci.update_cells(cell_list)

# Convert to a DataFrame and render.
cci = pd.DataFrame.from_records(rows[1:], columns=rows[0])

corr = mbc_df['mb_bf_disc_win_total'].corr(cci['cci'].astype(float))

#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=cci["Month"], y = cci["cci"], mode='lines', name="Centaline index"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   

fig.add_trace(go.Indicator( mode = "number",
    value = corr,
    title = {"text": "Correlation"},
    domain = {'y': [0,0], 'x': [0,0]}))


#Plot format

# Add figure title
fig.update_layout( xaxis_range=['1998-9','2021-3'],
    title_text="CCI VS MB"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>CCI</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_traces(number_font_size=50, selector=dict(type='indicator'))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.show()


In [6]:
#@title #5 - MB vs HIBOR
HIBOR = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-endperiod?sortby=end_of_month&sortorder=asc'

#Getting data from 6.3.1 (Hong Kong Interbank Interest Rates – End of period figures) and store it as a pandas dataframe type
HIBOR_df = pd.json_normalize(get_data_from_HKMA(HIBOR))


corr = mbc_df['mb_bf_disc_win_total'].corr(HIBOR_df['ir_1m'][26:])


#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   




# Set x-axis title
fig.update_xaxes(title_text="end_of_month")




# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

fig.add_trace(go.Indicator( mode = "number",
    value = corr,
    title = {"text": "Correlation"},
    ))


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_traces(number_font_size=50, selector=dict(type='indicator'))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([   
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",
        autorange = True
          # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
        autorange = True
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.show()

 


In [7]:
#@title ##6 -  AB vs HIBOR
#Plot 
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['aggr_balance_bf_disc_win'], mode='lines', name="AB"),secondary_y=True,) 

corr = mbc_df['aggr_balance_bf_disc_win'].corr(HIBOR_df['ir_1m'][26:])


#Plot format
# Add figure title
fig.update_layout( xaxis_range=['1998-7','2021-03'],title_text="AB VS HIBOR")

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

fig.add_trace(go.Indicator( mode = "number",
    value = corr,
    title = {"text": "Correlation"},
    ))


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))


fig.update_traces(number_font_size=50, selector=dict(type='indicator'))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([
          dict(count=1, label="1Months", step="month", stepmode="backward"),
          dict(count=3, label="3Months", step="month", stepmode="backward"),     
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",
        autorange = True
          # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
        autorange = True
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.show()

 


In [8]:
#@title #7 -  FPS Transactions and Registrations
fps_val_hkd = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-hkd-payment-val?sortby=end_of_month&sortorder=asc'
#Getting data from 3.12.4 (Clearing House Statistics – Turnover of HKD FPS payment (Value)) and store it as a pandas dataframe type
fps_val_hkd_df = pd.json_normalize(get_data_from_HKMA(fps_val_hkd))

fps_reg = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-fps-reg?sortby=end_of_month&sortorder=asc'
#Getting data from 3.12.2	(Faster Payment System Addressing Service Registrations) and store it as a pandas dataframe type
fps_reg_df = pd.json_normalize(get_data_from_HKMA(fps_reg))

#Plot
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#total - Real time credit transfer payment Total real time credit transfer payment + Real time direct debit payment + Batch mode payment
fig.add_trace(go.Bar(x=fps_val_hkd_df['end_of_month'], y = fps_val_hkd_df['total'], name="FPS Turnover of HKD FPS payment (Value in HK$ thousand)"),secondary_y=False)   

#total_grand_total - Total Grand-total
fig.add_trace(go.Scatter(x=fps_reg_df['end_of_month'], y = fps_reg_df['total_grand_total'], mode='lines+markers', name="FPS Addressing Service Registrations Total Grand-total"),secondary_y=True,)  

#Plot format
# Add figure title
fig.update_layout( xaxis_range=['2018-10','2021-03'],
    title_text="Transactions and Registrations"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>FPS Turnover of HKD FPS payment (Value in HK$ thousand)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>FPS Addressing Service Registrations Total Grand-total</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([    
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",
        autorange = True
          # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
        autorange = True
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.update_layout(legend= {'itemsizing': 'constant'})

fig.show()

 


In [9]:
#@title # 8 - China Total Export & Import Values

#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
#Source: http://www.customs.gov.cn/customs/syx/index.html
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Format the Dashboard_CCI URL of Googler Sheet
#https://docs.google.com/spreadsheets/d/13P9RLWitu7aE31F11VolXtgCfvM9ZuR9ayBkUpV_3c8/edit#gid=0
wb = gc.open_by_key('13P9RLWitu7aE31F11VolXtgCfvM9ZuR9ayBkUpV_3c8')

#Open the chiTrade worksheet
ws_chiTrade = wb.worksheet('chiTrade')
# get_all_values gives a list of rows.
rows = ws_chiTrade.get_all_values()

# Convert to a DataFrame and render.
chiTrade = pd.DataFrame.from_records(rows[1:], columns=rows[0])


#Plot
# Create figure with secondary y-axis

fig = go.Figure()

fig.add_trace(go.Scatter(x=chiTrade["Year&Month"], y = chiTrade["Export"], mode='lines', name="Export"))   

fig.add_trace(go.Scatter(x=chiTrade["Year&Month"], y = chiTrade["Import"], mode='lines', name="Import"))   


#Plot format

# Add figure title
fig.update_layout( xaxis_range=['2013-1','2020-01'],
    title_text="China Trade"
)

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="<b>RMB$ 100 million</b>")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_xaxes(
  rangeselector=dict(
      buttons=list([    
          dict(count=1, label="1y", step="year", stepmode="backward"),
          dict(count=3, label="3y", step="year", stepmode="backward"),
          dict(count=6, label="6y", step="year", stepmode="backward"),
          dict(count=10, label="10y", step="year", stepmode="backward"),
          dict(step="all")
      ])
    )
)

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",
        autorange = True
          # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
        autorange = True
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

fig.show()


In [10]:
#@title #9 - China Capital Flows *

#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Format the Dashboard_CCI URL of Googler Sheet
#https://docs.google.com/spreadsheets/d/1IZHS4Q_Q6hvwPhZ4OGZ2mgvYfiMDz8HBqwQqQ5aJIlk/edit#gid=0
wb = gc.open_by_key('1IZHS4Q_Q6hvwPhZ4OGZ2mgvYfiMDz8HBqwQqQ5aJIlk')

#Open the chiTrade worksheet
ws_chiFlow = wb.worksheet('chiFlow')
# get_all_values gives a list of rows.
rows = ws_chiFlow.get_all_values()

# Convert to a DataFrame and render.
chiFlow = pd.DataFrame.from_records(rows[1:], columns=rows[0])


chiFlow_dfl = pd.melt(chiFlow, id_vars = ['Quarter'], value_vars =['Current Account', 'Financial and Capital Account' , 'Balacning Item'],var_name='Account', value_name='RMB$ 100million' )

#Plot
fig = px.area(chiFlow_dfl, x='Quarter' , y='RMB$ 100million', color = 'Account')

#Plot Format
fig.update_layout(title_text="China's Balance of Payments")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(legend= {'itemsizing': 'constant'})

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)


fig.show()


# #@title #9 - China Capital Flows *
# !wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/chinaFlow.csv
# #Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
# #Source: http://www.safe.gov.cn/safe/2019/0627/13519.html
#
# chiFlow = pd.read_csv('chinaFlow.csv')
#
# y = chiFlow['Total'].str.strip()
# y.replace(',','', regex=True, inplace=True)
# y = y.dropna()
# y = y.astype(int)
# color=np.array(['rgb(255,255,255)']*y.shape[0])
# color[y<0]='rgb(255,0,0)'
# color[y>=0]='rgb(0,255,0)'
#
# #Plot
#
# fig = go.Figure()
#
# fig.add_trace(
#     go.Bar(name='Capital Flows',
#            x=chiFlow['Year&Month'],
#            y=chiFlow['Total'],
#            marker_color=color))
#
# #fig = px.bar(chiFlow, x='Year&Month', y='Total', title="Capital Flows", marker_color=color)
#
#
#
#
#
# #Plot format
#
# # Add figure title
# fig.update_layout( xaxis_range=['2010-Q1','2020-Q3'],
#     title_text="China Capital Flows (using the difference between Capital and Financial Account Balance of the Balance of Payments")
#
# # Set x-axis title
# fig.update_xaxes(title_text="Quarter")
#
# # Set y-axes titles
# fig.update_yaxes(title_text="<b>RMB$ 100 million</b>")
#
# fig.update_xaxes(
#         showgrid=False,  # Removes X-axis grid lines
#         linecolor="#BCCCDC",  # Sets color of X-axis line
# )
#
# fig.update_yaxes(
#         showgrid=False,  # Removes Y-axis grid lines
#         linecolor="#BCCCDC",  # Sets color of X-axis line
# )
#
# fig.update_layout(
#     plot_bgcolor="#FFF",  # Sets background color to white
# )
#
#
# fig.show()
#
# #!wget -O 'chiFlow_2.xlsx' 'http://www.safe.gov.cn/safe/file/file/20201225/d8bc5ca4ba1f4d178ce11f9fe4edb20b.xlsx?n=%E4%B8%AD%E5%9B%BD%E5%9B%BD%E9%99%85%E6%94%B6%E6%94%AF%E5%B9%B3%E8%A1%A1%E8%A1%A8%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(BPM6)'
# # temp_rmb = pd.read_excel('chiFlow_2.xlsx', sheet_name='季度BOP（人民币）')
# # temp_usd = pd.read_excel('chiFlow_2.xlsx', sheet_name='季度BOP（美元）')
# # temp_sdr = pd.read_excel('chiFlow_2.xlsx', sheet_name='季度BOP（SDR）')
# # ChiFlow = pd.DataFrame(([temp_rmb.iloc[73],temp_usd.iloc[73], temp_sdr.iloc[73]]),columns=['RMB', 'USD', 'SGD'] )
#
# # ChiFlow = ChiFlow.T
#
# # ChiFlow['CNY'] * 0.154664933
# # !pip install forex-python
# # from forex_python.converter import CurrencyRates
# # c = CurrencyRates()
# # c.get_rate('CNY', 'USD')
#
#






In [11]:
#@title #10 - HK Trade (To exclude China Clicking the THE MAINLAND OF CHINA legend)
#!wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/HKTopTotal.csv
#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
#Source: https://www.censtatd.gov.hk/hkstat/sub/so230_tc.jsp


#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Format the Dashboard_CCI URL of Googler Sheet
#https://docs.google.com/spreadsheets/d/1eRzmV5VdU48HbR0yHkog6fo3VpkDwGjuGbkp6m5_jSI/edit#gid=0
wb = gc.open_by_key('1eRzmV5VdU48HbR0yHkog6fo3VpkDwGjuGbkp6m5_jSI')

#Open the chiTrade worksheet
ws_HKTopTotal= wb.worksheet('HKTopTotal')
# get_all_values gives a list of rows.
rows = ws_HKTopTotal.get_all_values()

# Convert to a DataFrame and render.
HKTopTotal = pd.DataFrame.from_records(rows[1:], columns=rows[0])




#Plot

fig = go.Figure()

chi = HKTopTotal.query("Consignment == 'THE MAINLAND OF CHINA'")
fig.add_trace(go.Scatter(x=chi["Year"], y = chi["Value"], mode='lines', name="THE MAINLAND OF CHINA"))   

us = HKTopTotal.query("Consignment == 'USA'")
fig.add_trace(go.Scatter(x=us["Year"], y = us["Value"], mode='lines', name="USA"))   

tw = HKTopTotal.query("Consignment == 'TAIWAN'")
fig.add_trace(go.Scatter(x=tw["Year"], y = tw["Value"], mode='lines', name="TAIWAN")) 

jp = HKTopTotal.query("Consignment == 'JAPAN'")
fig.add_trace(go.Scatter(x=jp["Year"], y = jp["Value"], mode='lines', name="JAPAN")) 

sg = HKTopTotal.query("Consignment == 'SINGAPORE'")
fig.add_trace(go.Scatter(x=sg["Year"], y = sg["Value"], mode='lines', name="SINGAPORE")) 

kr = HKTopTotal.query("Consignment == 'KOREA'")
fig.add_trace(go.Scatter(x=kr["Year"], y = kr["Value"], mode='lines', name="KOREA")) 

my = HKTopTotal.query("Consignment == 'MALAYSIA'")
fig.add_trace(go.Scatter(x=my["Year"], y = my["Value"], mode='lines', name="MALAYSIA")) 

ind = HKTopTotal.query("Consignment == 'INDIA'")
fig.add_trace(go.Scatter(x=ind["Year"], y = ind["Value"], mode='lines', name="INDIA")) 

vm = HKTopTotal.query("Consignment == 'VIET NAM'")
fig.add_trace(go.Scatter(x=vm["Year"], y = vm["Value"], mode='lines', name="VIET NAM")) 

th = HKTopTotal.query("Consignment == 'TAIWAN'")
fig.add_trace(go.Scatter(x=th["Year"], y = th["Value"], mode='lines', name="TAIWAN")) 

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

# Set x-axis title
fig.update_xaxes(title_text="Total trade by Main Consignment, 2012 - 2019")

# Set y-axes titles
fig.update_yaxes(title_text="<b>HKD$ 100 million</b>")




In [12]:
#@title #11- HK Imports (To exclude China Clicking the THE MAINLAND OF CHINA legend)
#!wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/HKTopImports.csv
#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
#Source: https://www.censtatd.gov.hk/hkstat/sub/so230_tc.jsp


#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Format the Dashboard_CCI URL of Googler Sheet
#https://docs.google.com/spreadsheets/d/1Jxc-cGbN_CIRlO8JUc0jYPhTXjUVjVkzthUOqV7Qsec/edit#gid=0
wb = gc.open_by_key('1Jxc-cGbN_CIRlO8JUc0jYPhTXjUVjVkzthUOqV7Qsec')

#Open the chiTrade worksheet
ws_HKTopImports= wb.worksheet('HKTopImports')
# get_all_values gives a list of rows.
rows = ws_HKTopImports.get_all_values()

# Convert to a DataFrame and render.
HKTopImports = pd.DataFrame.from_records(rows[1:], columns=rows[0])


#Plot 

fig = go.Figure()


chi = HKTopImports.query("Origin == 'THE MAINLAND OF CHINA'")
fig.add_trace(go.Scatter(x=chi["Year"], y = chi["Value"], mode='lines', name="THE MAINLAND OF CHINA"))   

us = HKTopImports.query("Origin == 'USA'")
fig.add_trace(go.Scatter(x=us["Year"], y = us["Value"], mode='lines', name="USA"))   

tw = HKTopImports.query("Origin == 'TAIWAN'")
fig.add_trace(go.Scatter(x=tw["Year"], y = tw["Value"], mode='lines', name="TAIWAN")) 

jp = HKTopImports.query("Origin == 'JAPAN'")
fig.add_trace(go.Scatter(x=jp["Year"], y = jp["Value"], mode='lines', name="JAPAN")) 

sg = HKTopImports.query("Origin == 'SINGAPORE'")
fig.add_trace(go.Scatter(x=sg["Year"], y = sg["Value"], mode='lines', name="SINGAPORE")) 

kr = HKTopImports.query("Origin == 'KOREA'")
fig.add_trace(go.Scatter(x=kr["Year"], y = kr["Value"], mode='lines', name="KOREA")) 

my = HKTopImports.query("Origin == 'MALAYSIA'")
fig.add_trace(go.Scatter(x=my["Year"], y = my["Value"], mode='lines', name="MALAYSIA")) 

ind = HKTopImports.query("Origin == 'INDIA'")
fig.add_trace(go.Scatter(x=ind["Year"], y = ind["Value"], mode='lines', name="INDIA")) 

vm = HKTopImports.query("Origin == 'VIET NAM'")
fig.add_trace(go.Scatter(x=vm["Year"], y = vm["Value"], mode='lines', name="VIET NAM")) 

th = HKTopImports.query("Origin == 'TAIWAN'")
fig.add_trace(go.Scatter(x=th["Year"], y = th["Value"], mode='lines', name="TAIWAN")) 

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

# Set x-axis title
fig.update_xaxes(title_text="Total Imports by Main Origins, 2012 - 2019")

# Set y-axes titles
fig.update_yaxes(title_text="<b>HKD$ 100 million</b>")

In [13]:
#@title #12 - HK Exports (To exclude China Clicking the THE MAINLAND OF CHINA legend)
#!wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/HKTopExports.csv
#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
#Source: https://www.censtatd.gov.hk/hkstat/sub/so230_tc.jsp

#Getting data from BruceAUHK GitHub(Hv to manually update the lastest data monthly) and store it as a pandas dataframe type
gc = gspread.authorize(GoogleCredentials.get_application_default())

#Format the Dashboard_CCI URL of Googler Sheet
#https://docs.google.com/spreadsheets/d/1sSRnnKl3-9vLngQbVqs4U5gQxSKbBCBUDtD-IuOD9Ig/edit#gid=0
wb = gc.open_by_key('1sSRnnKl3-9vLngQbVqs4U5gQxSKbBCBUDtD-IuOD9Ig')

#Open the chiTrade worksheet
ws_HKTopExports= wb.worksheet('HKTopExports')
# get_all_values gives a list of rows.
rows = ws_HKTopExports.get_all_values()

# Convert to a DataFrame and render.
HKTopExports = pd.DataFrame.from_records(rows[1:], columns=rows[0])


#Plot

fig = go.Figure()

chi = HKTopTotal.query("Consignment == 'THE MAINLAND OF CHINA'")
fig.add_trace(go.Scatter(x=chi["Year"], y = chi["Value"], mode='lines', name="THE MAINLAND OF CHINA")) 

us = HKTopExports.query("Consignment == 'USA'")
fig.add_trace(go.Scatter(x=us["Year"], y = us["Value"], mode='lines', name="USA"))   

tw = HKTopExports.query("Consignment == 'TAIWAN'")
fig.add_trace(go.Scatter(x=tw["Year"], y = tw["Value"], mode='lines', name="TAIWAN")) 

jp = HKTopExports.query("Consignment == 'JAPAN'")
fig.add_trace(go.Scatter(x=jp["Year"], y = jp["Value"], mode='lines', name="JAPAN")) 

sg = HKTopExports.query("Consignment == 'SINGAPORE'")
fig.add_trace(go.Scatter(x=sg["Year"], y = sg["Value"], mode='lines', name="SINGAPORE")) 

kr = HKTopExports.query("Consignment == 'KOREA'")
fig.add_trace(go.Scatter(x=kr["Year"], y = kr["Value"], mode='lines', name="KOREA")) 

my = HKTopExports.query("Consignment == 'MALAYSIA'")
fig.add_trace(go.Scatter(x=my["Year"], y = my["Value"], mode='lines', name="MALAYSIA")) 

ind = HKTopExports.query("Consignment == 'INDIA'")
fig.add_trace(go.Scatter(x=ind["Year"], y = ind["Value"], mode='lines', name="INDIA")) 

vm = HKTopExports.query("Consignment == 'VIET NAM'")
fig.add_trace(go.Scatter(x=vm["Year"], y = vm["Value"], mode='lines', name="VIET NAM")) 

th = HKTopExports.query("Consignment == 'TAIWAN'")
fig.add_trace(go.Scatter(x=th["Year"], y = th["Value"], mode='lines', name="TAIWAN")) 

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

# Set x-axis title
fig.update_xaxes(title_text="Total trade by Main Consignment, 2012 - 2019")

# Set y-axes titles
fig.update_yaxes(title_text="<b>HKD$ 100 million</b>")




In [14]:
#@title # 13 - RMB’s (CNY) share as a global payments currency
def download_file(url):
  local_filename = url.split('/')[-1] + '.pdf'

  with requests.get(url) as r:
    with open(local_filename,'wb') as f:
      f.write(r.content)

    return local_filename

rmb_url = 'https://www.swift.com/swift-resource/250536/download'
rmb = download_file(rmb_url)

with pdfplumber.open(rmb) as pdf:
  page = pdf.pages[2]
  text = page.extract_text()

currency_df = DataFrame(re.findall(r'([A-Z]{3})\s(\d{1,2}\.\d{1,2})', text))

#fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])

fig = px.pie(currency_df, values=1, names=0 ,title='RMB Tracker Jan 2020')


fig.update_traces(textposition='inside', textinfo='value+label')

fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))



fig.show()

In [15]:
#@title #14 -  China Foreign currency reserves

#Official reserve assets 2021
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20210319/600adbbebdb44edd8c51c5bf62cf127f.xls?n=xlsx -O chiReserve21.xlsx &> /dev/null
#Official reserve assets 2020
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20210111/4eaceb5662ac420cb5e9a9b4e83c1877.xls?n=xlsx -O chiReserve20.xlsx &> /dev/null
#Official reserve assets 2019
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20200109/ac7f28cce4e640d1a1689e998c2efc40.xls?n=xlsx -O chiReserve19.xlsx &> /dev/null
#Official reserve assets 2018
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20190123/5f07d5ff59c140ea903b1a4332de8f55.xls?n=xlsx -O chiReserve18.xlsx &> /dev/null
#Official reserve assets 2017
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20180228/c64b8d713c5b4c24af9244e79565e7ef.xlsx?n=%E5%AE%98%E6%96%B9%E5%84%B2%E5%82%99%E8%B3%87%E7%94%A2%EF%BC%882017%EF%BC%89 -O chiReserve17.xlsx &> /dev/null
#Official reserve assets 2016
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20180112/c4ed8e83e0bf433ba8c81c1586025860.xlsx?n=%E5%AE%98%E6%96%B9%E5%84%B2%E5%82%99%E8%B3%87%E7%94%A2%EF%BC%882016%EF%BC%89 -O chiReserve16.xlsx &> /dev/null
#Official reserve assets 2015
!wget https://www.safe.gov.cn/big5/big5/www.safe.gov.cn:443/safe/file/file/20180112/3a8ada9cec94480f9da10fc6a3f793c2.xls?n=%E5%AE%98%E6%96%B9%E5%84%B2%E5%82%99%E8%B3%87%E7%94%A2%EF%BC%882015.06-2015.12%EF%BC%89 -O chiReserve15.xlsx &> /dev/null


chiReserve21 = pd.read_excel('chiReserve21.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve21.shape[0]))  # gives number of row count
index_list = [3,7]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve21 = pd.read_excel('chiReserve21.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)

chiReserve20 = pd.read_excel('chiReserve20.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve20.shape[0]))  # gives number of row count
index_list = [2,5]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve20 = pd.read_excel('chiReserve20.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)

#Official reserve assets 2019
chiReserve19 = pd.read_excel('chiReserve19.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve19.shape[0]))  # gives number of row count
index_list = [3,7]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve19 = pd.read_excel('chiReserve19.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)

#Official reserve assets 2018
chiReserve18 = pd.read_excel('chiReserve18.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve18.shape[0]))  # gives number of row count
index_list = [3,7]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve18 = pd.read_excel('chiReserve18.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)


#Official reserve assets 2017
chiReserve17 = pd.read_excel('chiReserve17.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve17.shape[0]))  # gives number of row count
index_list = [3,7]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve17 = pd.read_excel('chiReserve17.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)


#Official reserve assets 2017
chiReserve17 = pd.read_excel('chiReserve17.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve17.shape[0]))  # gives number of row count
index_list = [3,7]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve17 = pd.read_excel('chiReserve17.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)

#Official reserve assets 2016
chiReserve16 = pd.read_excel('chiReserve16.xlsx', usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)
num_lines  = list(range(chiReserve16.shape[0]))  # gives number of row count
index_list = [1,3]
to_exclude = [i for i in num_lines if i  not in index_list]
to_exclude
chiReserve16 = pd.read_excel('chiReserve16.xlsx', skiprows = to_exclude, usecols=[1,3,5,7,9,11,13,15,17,19,21,23],header=None)


from datetime import datetime

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month


d1 = datetime(2016, 1, 1)
month_counts = abs(diff_month(d1, datetime.today()))

chiReserve = pd.concat([chiReserve16,chiReserve17,chiReserve18,chiReserve19,chiReserve20, chiReserve21], axis=1)
chiReserve_transposed = chiReserve.T
chiReserve_transposed = chiReserve_transposed.rename(columns={0: "end_of_month", 1: "Reserve"})
chiReserve_transposed = chiReserve_transposed.reset_index(drop=True)
chiReserve_transposed['end_of_month'] = pd.Series(pd.date_range("2016-01-01", periods=month_counts, freq="M"))

fig = go.Figure()

fig.add_trace(go.Scatter(x = chiReserve_transposed["end_of_month"], y = chiReserve_transposed["Reserve"], mode='lines', name="Export")) 

#Plot format

# Add figure title
fig.update_layout(title_text="China Foreign currency reserves"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

fig.update_xaxes(
        showgrid=False,  # Removes X-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_yaxes(
        showgrid=False,  # Removes Y-axis grid lines
        linecolor="#BCCCDC",  # Sets color of X-axis line
)

fig.update_layout(
    plot_bgcolor="#FFF",  # Sets background color to white
)

# Set y-axes titles
fig.update_yaxes(title_text="<b>100million USD</b>")